In [1]:
# imports
%reload_ext autoreload
%autoreload 2

from basic_utils import *

pd.set_option('display.max_rows', 20)

Loading config.json


In [2]:
dates = read_dates('quote')

In [3]:
UNIT_SCALE = 10**9
# tgt_date = dates[-1:]
tgt_date = ['2018-10-05'] # hardcoded for now
tgt_date

['2018-10-05']

### load files

In [4]:
latest_quotes = load_csvs('quote_consol', tgt_date)
profile = load_csvs('summary_detail', ['assetProfile'])
latest_keystats = load_csvs('summary_detail', ['defaultKeyStatistics/' + str(tgt_date[0])])
latest_finstats = load_csvs('summary_detail', ['financialData/' + str(tgt_date[0])])


Loading file quote/csv/2018-10-05
Loading file summary-categories/assetProfile
Loading file summary-categories/defaultKeyStatistics/2018-10-05
Loading file summary-categories/financialData/2018-10-05


In [5]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 24 columns):
address1                     379 non-null object
city                         380 non-null object
state                        320 non-null object
zip                          379 non-null object
country                      380 non-null object
phone                        373 non-null object
fax                          109 non-null object
website                      380 non-null object
industry                     380 non-null object
industrySymbol               334 non-null object
sector                       380 non-null object
longBusinessSummary          380 non-null object
fullTimeEmployees            374 non-null float64
auditRisk                    328 non-null float64
boardRisk                    328 non-null float64
compensationRisk             328 non-null float64
shareHolderRightsRisk        328 non-null float64
overallRisk                  328 non-null float64
governa

In [6]:
latest_quotes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 69 columns):
ask                                  385 non-null float64
askSize                              385 non-null int64
averageDailyVolume10Day              385 non-null int64
averageDailyVolume3Month             385 non-null int64
bid                                  385 non-null float64
bidSize                              385 non-null int64
bookValue                            384 non-null float64
currency                             385 non-null object
earningsTimestamp                    356 non-null float64
earningsTimestampEnd                 344 non-null float64
earningsTimestampStart               344 non-null float64
epsForward                           377 non-null float64
epsTrailingTwelveMonths              384 non-null float64
exchange                             385 non-null object
exchangeDataDelayedBy                385 non-null int64
exchangeTimezoneName                 

In [7]:
latest_keystats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 35 columns):
maxAge                          379 non-null int64
priceHint                       379 non-null int64
enterpriseValue                 379 non-null int64
forwardPE                       368 non-null float64
profitMargins                   379 non-null float64
floatShares                     379 non-null int64
sharesOutstanding               379 non-null int64
sharesShort                     374 non-null float64
sharesShortPriorMonth           374 non-null float64
sharesShortPreviousMonthDate    374 non-null float64
dateShortInterest               374 non-null float64
sharesPercentSharesOut          373 non-null float64
heldPercentInsiders             374 non-null float64
heldPercentInstitutions         374 non-null float64
shortRatio                      374 non-null float64
shortPercentOfFloat             344 non-null float64
beta                            364 non-null float64
book

In [8]:
latest_finstats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 32 columns):
maxAge                     381 non-null int64
currentPrice               381 non-null float64
targetHighPrice            375 non-null float64
targetLowPrice             375 non-null float64
targetMeanPrice            375 non-null float64
targetMedianPrice          375 non-null float64
recommendationMean         374 non-null float64
recommendationKey          381 non-null object
numberOfAnalystOpinions    375 non-null float64
totalCash                  378 non-null float64
totalCashPerShare          378 non-null float64
ebitda                     379 non-null float64
totalDebt                  378 non-null float64
quickRatio                 379 non-null float64
currentRatio               379 non-null float64
totalRevenue               375 non-null float64
debtToEquity               288 non-null float64
revenuePerShare            375 non-null float64
returnOnAssets             373 non

### overview of all the companies in the coverage universe

- [x] Produce a list with an overview of all the companies in the coverage universe (name, symbol, address, sector, industry, description, employees)

In [9]:
profile = profile.drop_duplicates(subset='symbol')

In [10]:
len(profile.symbol.unique())

380

In [11]:
len(profile), len(profile.symbol.unique())

(380, 380)

In [12]:
brief_cols = ['symbol', 'website', 'address', 'sector', 'fullTimeEmployees', 'longBusinessSummary']

p2 = profile.copy()
p2['address'] = p2.address1 + ', ' \
    + p2.city + ', ' + p2.state + ', ' \
    + p2.zip + ', ' + p2.country
p2['sector'] = p2['sector'] + ', ' + p2['industry']

In [13]:
long_tickers = ["TSLA", "TAL", "SYMC", "AMAT", "MU", "CELG", "BABA", "FB", 
                "EDU", "ALLY", "GOOG", "CFG", "WDC", "SFTBY", "LRCX", "BAC", 
                "WFC", "KEY", "GM", "SHY", "SRLN", "NXPI", "XRAY", "TCEHY"]

In [14]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.html.border', 0)

sorted_symbols = profile.sort_values(by=['sector','industry']).symbol.tolist()
# sorted_symbols = long_tickers

with open('company-overviews.html', mode='w') as f:
    for s in sorted_symbols:
        f.write(p2[p2.symbol == s][brief_cols].T.to_html())
        f.write('<p style="page-break-before: always">')
    f.close()

pd.set_option('display.max_colwidth', 50)

In [15]:
risk_cols = ['symbol', 'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk']
profile[risk_cols].sort_values(by=['shareHolderRightsRisk'], ascending=False).head(10)

,symbol,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk
294,RNG,1.0,4.0,8.0,10.0,9.0
21,AME,2.0,6.0,9.0,10.0,9.0
279,PSTG,2.0,9.0,9.0,10.0,10.0
24,ANET,2.0,10.0,10.0,10.0,10.0
256,NWSA,1.0,9.0,3.0,10.0,10.0
255,NWS,1.0,9.0,3.0,10.0,10.0
107,DSW,2.0,7.0,4.0,10.0,9.0
28,APPF,2.0,8.0,8.0,10.0,10.0
126,EXPE,2.0,10.0,10.0,10.0,10.0
241,NFLX,2.0,10.0,10.0,10.0,10.0


In [17]:
date_cols = ['governanceEpochDate', 'compensationAsOfEpochDate']
profile.fillna(0, inplace=True)
profile.governanceEpochDate.apply(lambda x: date.fromtimestamp(x)).value_counts()
profile.compensationAsOfEpochDate.apply(lambda x: date.fromtimestamp(x)).value_counts()

2016-12-30    291
2017-12-30     58
1969-12-31     26
2015-12-30      1
2013-12-30      1
2005-12-30      1
2010-12-30      1
2011-12-30      1
Name: compensationAsOfEpochDate, dtype: int64

- [x] What is the distribution (frequency) of companies by sector and industry?

In [18]:
profile.sector.value_counts()

Technology                160
Healthcare                114
Consumer Cyclical          54
Consumer Defensive         30
Industrials                16
Financial Services          5
Communication Services      1
Name: sector, dtype: int64

In [19]:
count_by_ind = profile.groupby(by=['sector', 'industry']).symbol.count().sort_index()
with open('universe.html', mode='w') as f:
    f.write(pd.DataFrame(count_by_ind).to_html())
    f.close()
count_by_ind

sector                  industry                           
Communication Services  Telecom Services                        1
Consumer Cyclical       Apparel Manufacturing                   2
                        Apparel Stores                          8
                        Auto Manufacturers                      5
                        Auto Parts                              7
                        Broadcasting - TV                       2
                        Footwear & Accessories                  5
                        Home Improvement Stores                 2
                        Leisure                                 8
                        Luxury Goods                            1
                                                               ..
Technology              Electronic Gaming & Multimedia          4
                        Health Information Services             6
                        Information Technology Services         3
                

### summary metrics by sector

#### columns selection

In [20]:
profile.columns
pf_cols = ['symbol', 'sector', 'industry']

In [21]:
latest_quotes.columns
qs_cols = ['marketCap', 'shortName', 'trailingPE', 'regularMarketPrice', 
           'trailingAnnualDividendYield', 'earningsTimestamp', 
           'earningsTimestampEnd', 'earningsTimestampStart', 'epsForward',
           'epsTrailingTwelveMonths', 'fiftyDayAverageChangePercent',
           'fiftyTwoWeekHighChangePercent', 'fiftyTwoWeekLowChangePercent',
           'priceToBook', 'twoHundredDayAverageChangePercent']

In [22]:
latest_keystats.columns
ks_cols = ['enterpriseValue', 'forwardPE', 'pegRatio', 
           'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange',
           'shortPercentOfFloat', 'netIncomeToCommon']

In [23]:
latest_finstats.columns
fs_cols = ['ebitda', 'totalCashPerShare', 'totalDebt', 'totalRevenue',  
           'returnOnAssets', 'returnOnEquity', 'grossProfits', 'freeCashflow',
           'operatingCashflow', 'earningsGrowth', 'revenueGrowth', 'revenuePerShare', 
           'grossMargins', 'ebitdaMargins', 'operatingMargins', 
           'profitMargins', 'targetMeanPrice', 'targetMedianPrice', 
           'numberOfAnalystOpinions']

In [24]:
len(profile), len(latest_quotes), len(latest_keystats), len(latest_finstats)

(380, 385, 379, 381)

In [25]:
merged = profile.merge(
    latest_quotes, on=['symbol'], how='inner', sort=False).merge(
    latest_keystats, on=['symbol'], how='inner', sort=False).merge(
    latest_finstats, on=['symbol'], how='inner', sort=False)

# removes all duplicate columns
merged.drop([x for x in merged.columns.tolist() if '_y' in x], axis=1, inplace=True)
merged.rename(index=str, columns={ x: x.split('_')[0] for x in merged.columns}, inplace=True)

merged = merged[pf_cols + qs_cols + ks_cols + fs_cols]

In [26]:
merged.columns

Index(['symbol', 'sector', 'industry', 'marketCap', 'shortName', 'trailingPE',
       'regularMarketPrice', 'trailingAnnualDividendYield',
       'earningsTimestamp', 'earningsTimestampEnd', 'earningsTimestampStart',
       'epsForward', 'epsTrailingTwelveMonths', 'fiftyDayAverageChangePercent',
       'fiftyTwoWeekHighChangePercent', 'fiftyTwoWeekLowChangePercent',
       'priceToBook', 'twoHundredDayAverageChangePercent', 'enterpriseValue',
       'forwardPE', 'pegRatio', 'enterpriseToEbitda', '52WeekChange',
       'SandP52WeekChange', 'shortPercentOfFloat', 'netIncomeToCommon',
       'ebitda', 'totalCashPerShare', 'totalDebt', 'totalRevenue',
       'returnOnAssets', 'returnOnEquity', 'grossProfits', 'freeCashflow',
       'operatingCashflow', 'earningsGrowth', 'revenueGrowth',
       'revenuePerShare', 'grossMargins', 'ebitdaMargins', 'operatingMargins',
       'profitMargins', 'targetMeanPrice', 'targetMedianPrice',
       'numberOfAnalystOpinions'],
      dtype='object')

#### Analytics

- [x] Sum key metrics by sector (market cap, revenues, income, cash flow, …) to understand  relevant size

In [27]:
# current date is dirty, new dates won't include these tickers
# merged[merged.symbol == 'ZG']
# merged.drop('185', inplace=True) # MTRX
# merged.drop('313', inplace=True) # Z
# merged.drop('315', inplace=True) # ZG
# merged.drop(['246','97','230','90'], inplace=True) # SAND, EMN, PX, DWDP
# merged.drop(['174','134','162'], inplace=True) # MGTI, HLAN, LC
# merged.drop(['48','234','53', '143'], inplace=True) # CALL, RDCM, CELL, IDCC
# merged.drop(['83','118','292', '178'], inplace=True) # DE, GE, UPS, MMM
# merged.drop(['249','47','286', '63'], inplace=True) # SHAK, CAKE, TXRH, CMG

# CRS, ATI, HRB, VAC, THO, PRLB, TREE, CVCO, KTOS, CSU
# merged.drop(['90','35', '173', '369', '340', '290', '345', '98', '209', '95'], inplace=True) 
# merged.drop(['70'], inplace=True) # CEL

# merged[merged.symbol != 'AAPL']
merged[merged.sector == 'Financial Services'].sort_values(by='marketCap')

,symbol,sector,industry,marketCap,shortName,trailingPE,regularMarketPrice,trailingAnnualDividendYield,earningsTimestamp,earningsTimestampEnd,...,earningsGrowth,revenueGrowth,revenuePerShare,grossMargins,ebitdaMargins,operatingMargins,profitMargins,targetMeanPrice,targetMedianPrice,numberOfAnalystOpinions
337,TSS,Financial Services,Credit Services,17486589952,"Total System Services, Inc.",27.240694,95.86,0.005370,1.540325e+09,1.540584e+09,...,0.250,-0.176,24.745,0.32041,0.22987,0.17240,0.14385,103.57,105.0,23.0
173,ICE,Financial Services,Financial Exchanges,43592679424,Intercontinental Exchange Inc.,17.790780,76.02,0.011593,1.540989e+09,1.525696e+09,...,0.100,0.056,8.161,1.00000,0.62432,0.53997,0.52819,83.67,83.0,15.0
282,PYPL,Financial Services,Credit Services,98457673728,"PayPal Holdings, Inc.",49.541393,83.18,NaN,1.539877e+09,1.540222e+09,...,0.294,0.230,12.140,0.46644,0.18162,0.14761,0.14024,97.98,100.0,42.0
208,MA,Financial Services,Credit Services,221459841024,Mastercard Incorporated,47.826866,213.26,0.004493,1.540903e+09,1.541162e+09,...,0.364,0.200,13.246,1.00000,0.58825,0.55464,0.33809,232.03,230.0,37.0
353,V,Financial Services,Credit Services,323441983488,Visa Inc.,35.488280,145.36,0.005621,1.540411e+09,1.540816e+09,...,0.162,0.148,8.881,0.96680,0.68267,0.66710,0.47908,162.35,162.5,34.0


In [28]:
# check for companies with less than 100M of revenue
merged[merged.marketCap < (0.5 * UNIT_SCALE)].sort_values(by=['sector','industry']).industry.value_counts()
# merged[merged.marketCap < (0.5 * UNIT_SCALE)].sort_values(by=['sector','industry']).marketCap / UNIT_SCALE
# merged[merged.totalRevenue < (0.1 * UNIT_SCALE)].sort_values(by=['sector','industry'])[['symbol', 'shortName', 'industry']]

Biotechnology                          16
Communication Equipment                 3
Semiconductor Equipment & Materials     2
Diversified Industrials                 1
Computer Systems                        1
Software - Application                  1
Diagnostics & Research                  1
Name: industry, dtype: int64

In [29]:
merged.industry.value_counts()

Biotechnology                     44
Software - Application            41
Semiconductors                    26
Software - Infrastructure         21
Diagnostics & Research            18
Communication Equipment           13
Internet Content & Information    12
Medical Instruments & Supplies    11
Specialty Retail                  11
Medical Devices                   11
                                  ..
Consumer Electronics               2
Apparel Manufacturing              2
Confectioners                      2
Semiconductor Memory               1
Food Distribution                  1
Financial Exchanges                1
Luxury Goods                       1
Pharmaceutical Retailers           1
Grocery Stores                     1
Telecom Services                   1
Name: industry, Length: 51, dtype: int64

In [30]:
merged.columns

Index(['symbol', 'sector', 'industry', 'marketCap', 'shortName', 'trailingPE',
       'regularMarketPrice', 'trailingAnnualDividendYield',
       'earningsTimestamp', 'earningsTimestampEnd', 'earningsTimestampStart',
       'epsForward', 'epsTrailingTwelveMonths', 'fiftyDayAverageChangePercent',
       'fiftyTwoWeekHighChangePercent', 'fiftyTwoWeekLowChangePercent',
       'priceToBook', 'twoHundredDayAverageChangePercent', 'enterpriseValue',
       'forwardPE', 'pegRatio', 'enterpriseToEbitda', '52WeekChange',
       'SandP52WeekChange', 'shortPercentOfFloat', 'netIncomeToCommon',
       'ebitda', 'totalCashPerShare', 'totalDebt', 'totalRevenue',
       'returnOnAssets', 'returnOnEquity', 'grossProfits', 'freeCashflow',
       'operatingCashflow', 'earningsGrowth', 'revenueGrowth',
       'revenuePerShare', 'grossMargins', 'ebitdaMargins', 'operatingMargins',
       'profitMargins', 'targetMeanPrice', 'targetMedianPrice',
       'numberOfAnalystOpinions'],
      dtype='object')

In [31]:
grouping = ['sector', 'industry']
display_scale = ['marketCap','totalRevenue','netIncomeToCommon', 'freeCashflow']
df = merged.groupby(by=grouping)[display_scale]
(df.sum().sort_values(by='freeCashflow', ascending=False) / UNIT_SCALE).round(2)

marketCap  totalRevenue  \
sector             industry                                                  
Healthcare         Drug Manufacturers - Major        1184.84        251.61   
Technology         Internet Content & Information    1792.20        242.78   
                   Semiconductors                    1118.91        230.37   
                   Consumer Electronics              1086.21        256.25   
Healthcare         Health Care Plans                  629.88        736.14   
Technology         Software - Infrastructure         1161.13        170.05   
Consumer Cyclical  Specialty Retail                  1473.06        404.01   
Healthcare         Biotechnology                      526.89         88.82   
Financial Services Credit Services                    660.85         53.03   
Consumer Defensive Discount Stores                    374.76        650.79   
...                                                      ...           ...   
Consumer Cyclical  Footwear & Accessories             144.03         53.17   
                   Broadcasting - TV                   15.69         18.05   
                   Apparel Manufacturing               16.52          8.84   
Technology         Health Information Services         55.50          8.37   
Consumer Cyclical  Luxury Goods                        14.93          4.42   
Industrials        Aerospace & Defense                 13.77          5.93   
Technology         Semiconductor Memory                50.54         30.39   
Healthcare         Medical Care                        77.26         75.21   
Consumer Defensive Education & Training Services       37.56          9.60   
Consumer Cyclical  Auto Manufacturers                 377.84        778.50   

                                                   netIncomeToCommon  \
sector             industry                                            
Healthcare         Drug Manufacturers - Major                  31.91   
Technology         Internet Content & Information              50.91   
                   Semiconductors                              47.86   
                   Consumer Electronics                        56.18   
Healthcare         Health Care Plans                           28.12   
Technology         Software - Infrastructure                   20.94   
Consumer Cyclical  Specialty Retail                            17.69   
Healthcare         Biotechnology                               10.69   
Financial Services Credit Services                             16.67   
Consumer Defensive Discount Stores                              8.36   
...                                                              ...   
Consumer Cyclical  Footwear & Accessories                       2.83   
                   Broadcasting - TV                           -3.03   
                   Apparel Manufacturing                        0.35   
Technology         Health Information Services                  1.11   
Consumer Cyclical  Luxury Goods                                 0.45   
Industrials        Aerospace & Defense                          0.61   
Technology         Semiconductor Memory                        14.14   
Healthcare         Medical Care                                 2.75   
Consumer Defensive Education & Training Services                1.18   
Consumer Cyclical  Auto Manufacturers                          36.77   

                                                   freeCashflow  
sector             industry                                      
Healthcare         Drug Manufacturers - Major             54.16  
Technology         Internet Content & Information         42.63  
                   Semiconductors                         42.52  
                   Consumer Electronics                   41.46  
Healthcare         Health Care Plans                      40.72  
Technology         Software - Infrastructure              38.33  
Consumer Cyclical  Specialty Retail                       29.73  
Healt

In [32]:
display_performance = ['fiftyDayAverageChangePercent', 
                       'twoHundredDayAverageChangePercent',
                       'fiftyTwoWeekHighChangePercent', 
                       'fiftyTwoWeekLowChangePercent']

df = merged.groupby(by='sector')[display_performance].mean()
# df.rank().mean(axis=1)
df.T

sector,Communication Services,Consumer Cyclical,Consumer Defensive,Financial Services,Healthcare,Industrials,Technology
fiftyDayAverageChangePercent,0.022042,-0.046135,-0.030518,-0.019129,-0.018923,0.014528,-0.052537
twoHundredDayAverageChangePercent,0.169954,-0.015556,-0.006605,0.049129,0.046082,0.112979,0.006885
fiftyTwoWeekHighChangePercent,-0.051700,-0.193925,-0.176596,-0.056660,-0.185368,-0.090119,-0.186693
fiftyTwoWeekLowChangePercent,0.377513,0.435015,0.263932,0.352772,0.545133,0.397701,0.447546


- [x] Show metrics of dispersion for key metrics by sector (same as above) to under ranges and define focus, limitations for research and investment parameters

In [33]:
from pandas import Series
# range_func = lambda x: Series([x.min(), x.mean(), x.median(), x.max()], index =['count', 'min', 'mean', 'median', 'max'])
df_range = merged.groupby(by='sector').marketCap.describe()
df_range[df_range.columns[1:]] = (df_range[df_range.columns[1:]] / UNIT_SCALE).round(1)
df_range

,count,mean,std,min,25%,50%,75%,max
sector,,,,,,,,
Communication Services,1.0,108.6,NaN,108.6,108.6,108.6,108.6,108.6
Consumer Cyclical,54.0,54.3,138.2,2.1,5.7,11.3,36.1,921.7
Consumer Defensive,30.0,44.9,61.2,2.8,12.1,23.5,41.4,273.3
Financial Services,5.0,140.9,128.8,17.5,43.6,98.5,221.5,323.4
Healthcare,113.0,33.1,58.0,0.0,1.5,10.2,39.6,373.2
Industrials,16.0,18.0,16.8,0.2,5.8,16.5,23.7,66.0
Technology,160.0,43.3,135.6,0.0,3.1,8.2,21.2,1083.3


In [34]:
df_range = merged.groupby(by='sector')[display_scale].describe()
df_range[df_range.columns[1:]] = (df_range[df_range.columns[1:]] / UNIT_SCALE).round(1)
df_range.T

sector                   Communication Services  Consumer Cyclical  \
marketCap         count                     1.0               54.0   
                  mean                    108.6               54.3   
                  std                       NaN              138.2   
                  min                     108.6                2.1   
                  25%                     108.6                5.7   
                  50%                     108.6               11.3   
                  75%                     108.6               36.1   
                  max                     108.6              921.7   
totalRevenue      count                     0.0                0.0   
                  mean                     84.0               30.6   
...                                         ...                ...   
netIncomeToCommon 75%                      12.0                1.2   
                  max                      12.0               22.9   
freeCashflow      count                     0.0                0.0   
                  mean                     17.1                0.9   
                  std                       NaN                3.7   
                  min                      17.1              -13.0   
                  25%                      17.1                0.1   
                  50%                      17.1                0.3   
                  75%                      17.1                0.9   
                  max                      17.1               14.0   

sector                   Consumer Defensive  Financial Services  Healthcare  \
marketCap         count                30.0                 5.0       113.0   
                  mean                 44.9               140.9        33.1   
                  std                  61.2               128.8        58.0   
                  min                   2.8                17.5         0.0   
                  25%                  12.1                43.6         1.5   
                  50%                  23.5                98.5        10.2   
                  75%                  41.4               221.5        39.6   
                  max                 273.3               323.4       373.2   
totalRevenue      count                 0.0                 0.0         0.0   
                  mean                 44.0                11.6        18.0   
...                                     ...                 ...         ...   
netIncomeToCommon 75%                   2.8                 4.7         0.8   
                  max                  10.7                 9.3        22.5   
freeCashflow      count                 0.0                 0.0         0.0   
                  mean                  1.8                 4.1         1.5   
                  std                   3.4                 4.3         3.2   
                  min                  -0.7                 0.5        -4.7   
                  25%                   0.4                 0.6        -0.0   
                  50%                   0.8                 2.3         0.3   
                  75%                   1.7                 6.9         1.5   
                  max                  17.3                10.3        18.0   

sector                   Industrials  Technology  
marketCap         count         16.0       160.0  
                  mean          18.0        43.3  
                  std           16.8       135.6  
                  min            0.2         0.0  
                  25%            5.8         3.1  
                  50%           16.5         8.2  
                  75%           23.7        21.2  
                  max           66.0      1083.3  
totalRevenue      count          0.0         0.0  
                  mean           4.7         8.8  
...                              ...         ...  
netIncomeToCommon 75%            0.9         0.5  
                  max            1.6        56.1  
freeCa

In [35]:
sector_des = merged[merged.sector == 'Technology'][display_scale].describe().T
sector_des[sector_des.columns[1:]] = (sector_des[sector_des.columns[1:]] / UNIT_SCALE).round(1)
sector_des

,count,mean,std,min,25%,50%,75%,max
marketCap,160.0,43.3,135.6,0.0,3.1,8.2,21.2,1083.3
totalRevenue,160.0,8.8,26.3,0.0,0.5,1.4,4.4,255.3
netIncomeToCommon,160.0,1.4,5.5,-4.2,-0.0,0.1,0.5,56.1
freeCashflow,153.0,1.4,4.6,-0.5,0.0,0.1,0.7,41.4


In [36]:
display_scale

['marketCap', 'totalRevenue', 'netIncomeToCommon', 'freeCashflow']

In [37]:
# financial metrics for companies are not correctly calculated, see FB and MU
sector_des = merged[merged.sector == 'Technology'][['symbol', 'industry'] + display_scale]
sector_des[sector_des.columns[2:]]  = (sector_des[sector_des.columns[2:]] / UNIT_SCALE).round(1)
sector_des.sort_values(by='freeCashflow', ascending=False)

,symbol,industry,marketCap,totalRevenue,netIncomeToCommon,freeCashflow
1,AAPL,Consumer Electronics,1083.3,255.3,56.1,41.4
227,MSFT,Software - Infrastructure,859.8,110.4,16.6,25.1
149,GOOG,Internet Content & Information,807.6,123.9,16.3,21.7
129,FB,Internet Content & Information,454.2,48.5,19.1,11.3
264,ORCL,Software - Infrastructure,186.9,39.9,3.9,10.6
89,CSCO,Communication Equipment,220.0,49.3,0.1,10.2
335,TSM,Semiconductors,202.9,33.2,11.6,9.0
321,TCEHY,Internet Content & Information,362.1,42.4,12.2,8.3
38,AVGO,Semiconductors,101.0,20.2,11.7,7.2
180,INTC,Semiconductors,216.9,66.2,13.3,6.6


- [x] Display top winners and losers for the last 52 weeks

In [38]:
merged.columns

Index(['symbol', 'sector', 'industry', 'marketCap', 'shortName', 'trailingPE',
       'regularMarketPrice', 'trailingAnnualDividendYield',
       'earningsTimestamp', 'earningsTimestampEnd', 'earningsTimestampStart',
       'epsForward', 'epsTrailingTwelveMonths', 'fiftyDayAverageChangePercent',
       'fiftyTwoWeekHighChangePercent', 'fiftyTwoWeekLowChangePercent',
       'priceToBook', 'twoHundredDayAverageChangePercent', 'enterpriseValue',
       'forwardPE', 'pegRatio', 'enterpriseToEbitda', '52WeekChange',
       'SandP52WeekChange', 'shortPercentOfFloat', 'netIncomeToCommon',
       'ebitda', 'totalCashPerShare', 'totalDebt', 'totalRevenue',
       'returnOnAssets', 'returnOnEquity', 'grossProfits', 'freeCashflow',
       'operatingCashflow', 'earningsGrowth', 'revenueGrowth',
       'revenuePerShare', 'grossMargins', 'ebitdaMargins', 'operatingMargins',
       'profitMargins', 'targetMeanPrice', 'targetMedianPrice',
       'numberOfAnalystOpinions'],
      dtype='object')

In [39]:
cols = ['symbol', 'shortName', 'sector', 'industry', 'marketCap', 
        'fiftyTwoWeekLowChangePercent', 'enterpriseToEbitda', 'forwardPE', 'pegRatio']
# top / bottom 52wk performing by sectors & industries
merged.groupby(by=grouping).fiftyTwoWeekLowChangePercent.mean().sort_values()

sector              industry                           
Consumer Cyclical   Auto Manufacturers                     0.031041
                    Auto Parts                             0.056950
                    Broadcasting - TV                      0.079900
Consumer Defensive  Beverages - Soft Drinks                0.121258
Technology          Electronic Components                  0.150209
Consumer Defensive  Confectioners                          0.152612
Technology          Semiconductor Memory                   0.161514
Consumer Defensive  Packaged Foods                         0.167765
Financial Services  Financial Exchanges                    0.171160
Technology          Semiconductor Equipment & Materials    0.201876
                                                             ...   
Industrials         Aerospace & Defense                    0.588611
Healthcare          Health Care Plans                      0.611444
                    Medical Care                           0

In [40]:
# top / bottom 52wk performing by company
merged.sort_values(by='fiftyTwoWeekLowChangePercent')[cols]

,symbol,shortName,sector,industry,marketCap,fiftyTwoWeekLowChangePercent,enterpriseToEbitda,forwardPE,pegRatio
277,POWI,"Power Integrations, Inc.",Technology,Semiconductors,1715927552,0.002233,18.506,18.177570,2.26
220,MKSI,"MKS Instruments, Inc.",Technology,Scientific & Technical Instruments,4229888256,0.002988,6.761,8.935185,0.73
127,F,Ford Motor Company,Consumer Cyclical,Auto Manufacturers,36349857792,0.003300,11.908,6.656934,0.45
184,IPGP,IPG Photonics Corporation,Technology,Semiconductor Equipment & Materials,7137256960,0.003629,10.621,15.172571,1.65
355,VC,Visteon Corporation,Consumer Cyclical,Auto Parts,2598473472,0.003971,9.611,12.138546,0.94
60,BWA,BorgWarner Inc.,Consumer Cyclical,Auto Parts,8709754880,0.004093,6.293,8.797469,1.25
324,TEL,TE Connectivity Ltd. New Switze,Technology,Electronic Components,29336764416,0.004175,11.040,13.961857,1.50
111,EBAY,eBay Inc.,Consumer Cyclical,Specialty Retail,31903094784,0.004361,12.817,12.496125,1.00
103,DLPH,Delphi Technologies PLC Ordinar,Consumer Cyclical,Auto Parts,2309316096,0.005023,5.278,5.060311,1.83
11,AEIS,"Advanced Energy Industries, Inc",Technology,Electronic Components,1899356416,0.005383,6.597,9.540275,1.13


- [ ] Plot key return / performance metrics by market cap, revenues and cashflows

In [41]:
merged[merged.industry == 'Solar']

,symbol,sector,industry,marketCap,shortName,trailingPE,regularMarketPrice,trailingAnnualDividendYield,earningsTimestamp,earningsTimestampEnd,...,earningsGrowth,revenueGrowth,revenuePerShare,grossMargins,ebitdaMargins,operatingMargins,profitMargins,targetMeanPrice,targetMedianPrice,numberOfAnalystOpinions


In [42]:
merged.columns

Index(['symbol', 'sector', 'industry', 'marketCap', 'shortName', 'trailingPE',
       'regularMarketPrice', 'trailingAnnualDividendYield',
       'earningsTimestamp', 'earningsTimestampEnd', 'earningsTimestampStart',
       'epsForward', 'epsTrailingTwelveMonths', 'fiftyDayAverageChangePercent',
       'fiftyTwoWeekHighChangePercent', 'fiftyTwoWeekLowChangePercent',
       'priceToBook', 'twoHundredDayAverageChangePercent', 'enterpriseValue',
       'forwardPE', 'pegRatio', 'enterpriseToEbitda', '52WeekChange',
       'SandP52WeekChange', 'shortPercentOfFloat', 'netIncomeToCommon',
       'ebitda', 'totalCashPerShare', 'totalDebt', 'totalRevenue',
       'returnOnAssets', 'returnOnEquity', 'grossProfits', 'freeCashflow',
       'operatingCashflow', 'earningsGrowth', 'revenueGrowth',
       'revenuePerShare', 'grossMargins', 'ebitdaMargins', 'operatingMargins',
       'profitMargins', 'targetMeanPrice', 'targetMedianPrice',
       'numberOfAnalystOpinions'],
      dtype='object')

In [43]:
valuation_cols = ['forwardPE', 'pegRatio', 'enterpriseToEbitda']
cols = ['symbol', 'industry'] + valuation_cols
mask = (merged.forwardPE > 0) & (merged.forwardPE < 20) & (merged.pegRatio > 0) & (merged.pegRatio < 1)
df = merged[mask].groupby(by=['industry'])[valuation_cols]
df.mean().sort_values(by='pegRatio').round(1)

,forwardPE,pegRatio,enterpriseToEbitda
industry,,,
Semiconductor Memory,4.4,0.2,2.5
Semiconductor Equipment & Materials,7.2,0.4,5.6
Auto Manufacturers,6.2,0.5,9.2
Biotechnology,11.9,0.5,15.1
Education & Training Services,15.6,0.7,28.8
Auto Parts,11.6,0.7,10.3
Medical Care,12.9,0.7,9.8
Scientific & Technical Instruments,12.1,0.8,9.6
Drug Manufacturers - Major,10.6,0.8,13.6


In [44]:
merged[mask & (merged.industry == 'Apparel Stores')][cols].sort_values(by='pegRatio').round(1)

,symbol,industry,forwardPE,pegRatio,enterpriseToEbitda
152,GPS,Apparel Stores,10.2,0.9,5.5
350,URBN,Apparel Stores,13.9,1.0,8.4


### Old outputs

In [45]:
current_symbols = [
    "MU", "AMAT", "FB", "LRCX", "AEIS", "THO", "EXEL", "MKSI", "IPGP", "ANET",
    "HALO", "SUPN", "F", "LOGM", "NVDA", "OLED", "GSKY", "REGN", "CI", "HRB",
    "UBNT", "SWKS", "UTHR", "BKNG", "TXN", "STMP", "IONS", "AAPL", "YELP",
    "RHT", "ORCL", "KLAC", "GM", "ROG", "WCG", "CSU", "TER", "SKX", "VAC",
    "MA", "V", "CTSH", "FL", "EXPE", "ADBE", "MSFT", "FSLR", "BRKS", "CGNX",
    "ULTA", "CRUS", "BMY", "BBY", "FFIV", "GNTX", "PLCE", "GOOG", "EA", "GILD",
    "HUM", "ADP", "MXIM", "CERN", "MOH", "CNC", "ANTM", "FTNT", "CSCO", "DECK",
    "IRBT", "PAYX", "INTU", "TJX", "JNPR", "GPS", "CCMP", "ROST", "ATVI",
    "EBS", "ICUI", "BC", "AVX", "PANW", "XLNX", "EW", "VRTX", "PEGA", "NTAP",
    "POWI", "DOX", "NKE", "EEFT", "MANH", "ETSY", "LGND", "NTGR", "VC", "ABMD",
    "SHOO", "CA", "CRM", "MRVL", "AEO", "DSW", "CBM", "VAR", "GHC", "LOPE",
    "QCOM", "TWTR", "EPAM", "NATI", "SLAB", "A", "DLB", "CDNS", "ZNGA", "GRPN",
    "RL", "TTD", "QLYS", "ANSS", "WAT", "VEEV", "URBN", "OLLI", "COLM", "TIF",
    "NWS", "PLT", "FIVE", "GMED", "NWSA", "CVCO", "MASI", "DBX", "AKAM",
    "ISRG", "BRKR", "ALGN", "PCTY", "FLIR", "DORM", "WWE", "LULU", "PAYC",
    "TYL", "GRUB", "HQY", "EXLS", "APPF", "VSH", "MYGN", "MDSO", "TTWO",
    "ALRM", "SNPS", "ULTI", "NOW", "ATGE", "ILMN", "SMTC", "ELLI", "TDC",
    "MPWR", "MIME", "NEOG", "TRIP", "WST", "MSG", "SPLK", "CVLT", "RGEN",
    "CSOD", "HUBS", "NEWR", "GWRE", "CHGG", "DXCM", "FIVN", "SQ", "RNG",
    "DATA", "WDAY", "PSTG", "COUP", "BL", "CREE", "BOX", "AYX", "ZEN", "ADSK",
    "AMD", "AME", "AMZN", "APTV", "AQB", "ARCT", "ARNC", "ATHN", "ATI", "ATTU",
    "AUDC", "AVAV", "BABA", "BGNE", "BIDU", "BIIB", "BLCM", "BLUE", "CAMT",
    "CEL", "CELG", "CERS", "CGEN", "CHKP", "CLLS", "CRNT", "CRS", "CRSP",
    "CYBR", "DDD", "DMLRY", "EDIT", "ESLT", "EVGN", "EDU", "TAL", "EVLO",
    "FARO", "FOMX", "GBTC", "GILT", "GLMD", "HDP", "HPQ", "HXL", "ICE",
    "IFNNY", "INCY", "INO", "IOVA", "ITRN", "JD", "KMDA", "KODK", "KTOS",
    "KTWO", "MCRB", "MDT", "MELI", "MLNX", "MTLS", "MZOR", "NFLX", "NICE",
    "NNDM", "NSTG", "NTEC", "NTLA", "NVMI", "NVTA", "NXPI", "ONCE", "ONVO",
    "OPRA", "ORBK", "PACB", "PNTR", "PRLB", "PSTI", "PTC", "PYPL", "RDHL",
    "RDWR", "RKUNY", "ROK", "RUBY", "SEDG", "SFTBY", "SHOP", "SPNS", "SPOT",
    "SSYS", "SYK", "SYRS", "TARO", "TCEHY", "TDOC", "TEVA", "TM", "TREE",
    "TRMB", "TSEM", "TSLA", "TSM", "TWOU", "UCTT", "URGN", "VCYT", "WIX",
    "XONE", "YY", "ZUO"]
new_sp_symbols = [
    "BF.A", "STZ", "TAP", "IDXX", "ZTS", "F", "GM", "BWA", "DLPH", "GPC", "LKQ", "ABBV", "ALXN",
    "AMGN", "BIIB", "BMY", "CELG", "GILD", "INCY", "NKTR", "REGN", "VRTX", "FFIV", "HRS", "JNPR",
    "MSI", "CCL", "NCLH", "RCL", "NTAP", "STX", "WDC", "DGX", "LH", "ABC", "CAH", "HSIC", "MCK",
    "AMZN", "NFLX", "CPB", "KO", "CAG", "GIS", "HSY", "HRL", "SJM", "K", "KDP", "MDLZ", "MNST",
    "PEP", "KHC", "TSN", "SYY", "GRMN", "AET", "ANTM", "CI", "CNC", "HUM", "UNH", "DVA", "EVHC",
    "HCA", "UHS", "IQV", "HD", "LOW", "ADM", "MKC", "EBAY", "PYPL", "TWTR", "VRSN", "GLW", "A",
    "ABMD", "ALGN", "BAX", "BDX", "BSX", "DHR", "EW", "HOLX", "ILMN", "ISRG", "MDT", "PKI", "RMD",
    "SYK", "COO", "VAR", "XRAY", "ZBH", "FIS", "FISV", "FLT", "GPN", "MA", "TSS", "V", "ABT", "AGN",
    "JNJ", "LLY", "MRK", "MYL", "PFE", "PRGO", "COST", "CVS", "KR", "WBA", "WMT", "MTD", "TMO",
    "WAT", "ADI", "AVGO", "FSLR", "INTC", "MCHP", "MU", "NVDA", "QRVO", "QCOM", "SWKS", "TXN",
    "XLNX", "AMAT", "CDNS", "KLAC", "SNPS", "APH", "FLIR", "TEL", "ADBE", "ADSK", "CTXS", "ORCL",
    "CRM", "ANSS", "BR", "INTU", "CA", "RHT", "ATVI", "EA", "TTWO", "ADP", "GOOG", "AAPL", "FB", "HPE",
    "HPQ", "IBM", "MSFT", "BKNG", "EXPE", "TRIP",]

In [46]:
set(new_sp_symbols).difference(set(current_symbols))

{'ABBV',
 'ABC',
 'ABT',
 'ADI',
 'ADM',
 'AET',
 'AGN',
 'ALXN',
 'AMGN',
 'APH',
 'AVGO',
 'BAX',
 'BDX',
 'BF.A',
 'BR',
 'BSX',
 'BWA',
 'CAG',
 'CAH',
 'CCL',
 'COO',
 'COST',
 'CPB',
 'CTXS',
 'CVS',
 'DGX',
 'DHR',
 'DLPH',
 'DVA',
 'EBAY',
 'EVHC',
 'FIS',
 'FISV',
 'FLT',
 'GIS',
 'GLW',
 'GPC',
 'GPN',
 'GRMN',
 'HCA',
 'HD',
 'HOLX',
 'HPE',
 'HRL',
 'HRS',
 'HSIC',
 'HSY',
 'IBM',
 'IDXX',
 'INTC',
 'IQV',
 'JNJ',
 'K',
 'KDP',
 'KHC',
 'KO',
 'KR',
 'LH',
 'LKQ',
 'LLY',
 'LOW',
 'MCHP',
 'MCK',
 'MDLZ',
 'MKC',
 'MNST',
 'MRK',
 'MSI',
 'MTD',
 'MYL',
 'NCLH',
 'NKTR',
 'PEP',
 'PFE',
 'PKI',
 'PRGO',
 'QRVO',
 'RCL',
 'RMD',
 'SJM',
 'STX',
 'STZ',
 'SYY',
 'TAP',
 'TEL',
 'TMO',
 'TSN',
 'TSS',
 'UHS',
 'UNH',
 'VRSN',
 'WBA',
 'WDC',
 'WMT',
 'XRAY',
 'ZBH',
 'ZTS'}

In [47]:
cols = ['symbol', 'fiftyTwoWeekHighChangePercent']
latest_quotes[cols].sort_values(by='fiftyTwoWeekHighChangePercent')[:-10].head(10)

,symbol,fiftyTwoWeekHighChangePercent
341,KDP,-0.820150
220,GBTC,-0.811439
315,DLPH,-0.747451
189,AQB,-0.689690
243,NNDM,-0.668394
281,UCTT,-0.650188
232,JD,-0.533938
359,NKTR,-0.529364
222,GLMD,-0.515891
270,SYRS,-0.509648


In [48]:
latest_quotes.fiftyDayAverageChangePercent.plot.hist(bins=10)